# Model Universes


Objective: To get a feel for how different choices of $\Omega_{m,0}$, $\Omega_{\Lambda,0}$, $\Omega_{r,0}$ affect the evolution and fate of a universe.  
1. Let's recreate plot 5.8 from Ryden. This shows $a(t)$ from $t = 1e-10 H_0^{-1}$ to the present day. Do we get the correct scaling behaviors in the radiation, matter, and $\Lambda$-dominated regimes?
2. When do we switch from each regime to the next? How long ago did the shifts occur? At what redshift?
3. What happens as we look forward in time? What choices of $\Omega_{m,0}$, $\Omega_{\Lambda,0}$, and $\Omega_{r,0}$ lead to big crunches or endless expansion?

First, let's load a couple useful packages

In [61]:
import numpy as np
from scipy.interpolate import interp1d

Step 1: Compute $\dot{a}(a,\Omega_i)$

In [36]:
def aDot(a,Omegas,Ws):
    # Compute the sum over components
    Val = 0.
    for i in range(len(Omegas)):
        Val += Omegas[i] * a**(-3 * (1 + Ws[i]))

    ## Add the curvature term
    
    Omega0 = 0.
    for Omega in Omegas:
        Omega0 += Omega

    Val += (1 - Omega0) / a**2
    
    ## Take a square root:
    Val = (Val)**0.5
    
    ## Multiply by a
    Val *= a
    
    return(Val)

Let's do a simple numerical integral. We know the value at a particular time ($a(t_0) = 1$). A simple first-order integration should be sufficient for our task. We'll have to do a pair of integrations since our known point is in the middle of our integration range. We should use a variable timestep since the timescale ($H^{-1}$) is changing with time.

In [16]:
def integrateFriedmann(Omegas,Ws,nT=int(1e4)):

    ForwardAs = [1.]
    ForwardTs = [0.]
    BackwardAs = [1.]
    BackwardTs = [0.]
    
    for i in range(nT):
        currentaDot = aDot(ForwardAs[-1],Omegas,Ws)
        foredT = 0.001 * (ForwardAs[-1] / currentaDot)
        ForwardAs.append(ForwardAs[-1] + currentaDot * foredT)
        ForwardTs.append(ForwardTs[-1] + foredT)


    for i in range(nT):
        # And now the backward step:
        currentaDot = aDot(BackwardAs[-1],Omegas,Ws)
        backdT = 0.01 * (BackwardAs[-1] / currentaDot)
        BackwardTs.append(BackwardTs[-1] - backdT)
        BackwardAs.append(BackwardAs[-1] - currentaDot * backdT)

    Ts = []
    As = []

    for i in range(1,len(BackwardTs)+1):
        Ts.append(BackwardTs[-i])
        As.append(BackwardAs[-i])
    for i in range(len(ForwardTs)):
        Ts.append(ForwardTs[i])
        As.append(ForwardAs[i])
        
    return(Ts,As)


Let's also compute the three equality times, where the energy density due to pairs of components are equal.

In [26]:
def equalityAs(Omegas, Ws,Names):
    labels = []
    As = []
    for i in range(len(Omegas)):
        for j in range(i+1,len(Omegas)):
            labels.append(Names[i] + ', ' + Names[j])
            As.append(np.exp((np.log(Omegas[i] / Omegas[j])) / (-3 * (Ws[j] - Ws[i]))))
    return(As,labels)

Let's load a couple packages for plotting. 

In [62]:
%matplotlib notebook
from ipywidgets import interact, interactive,  fixed
from IPython.display import clear_output, display, HTML

import matplotlib.pyplot as plt

Finally, let's make a plot. This will 

In [66]:
@interact(fracL=0.70, fracM=0.30, logfracR=-4,Omega0 = 1.0,radScale=0.15, matterScale = 0.80)
def plotAT(fracL=0.70, fracM=0.30, logfracR=-4,Omega0 = 1.0,radScale=0.15, matterScale = 0.80):

    OmegaScale = Omega0 / (fracL + fracM + 10**logfracR)
    Omegas = [fracL * OmegaScale, fracM * OmegaScale, OmegaScale * 10**logfracR]
    Ws = [-1., 0., 1./3.]
    Names = ["L","M","R"]

    Ts, As = integrateFriedmann(Omegas, Ws)
    aTfn = interp1d(As,Ts - np.min(Ts))
    EAs, Labels = equalityAs(Omegas, Ws, Names)


    fig, axes = plt.subplots()
    axes.plot(np.subtract(Ts,np.min(Ts)),As, label="$a(t)$")
    axes.plot(np.subtract(Ts,np.min(Ts)), radScale * np.subtract(Ts,np.min(Ts))**0.5, label="$\propto t^{1/2}$",alpha=0.5)
    axes.plot(np.subtract(Ts,np.min(Ts)), matterScale * np.subtract(Ts,np.min(Ts))**(2./3.), label="$\propto t^{2/3}$",alpha=0.5)

    for i in range(len(EAs)):
        axes.axhline(EAs[i],linestyle=':')
        axes.axvline(aTfn(EAs[i]),linestyle=':')
        
    axes.axhline(1.0,linestyle=':')
    axes.axvline(1.0,linestyle=':')

    axes.set_yscale('log')
    axes.set_xscale('log')
    axes.set_ylim(1e-7,1e2)
    axes.set_xlim(1e-10,1e1)
    axes.set_xlabel("H_0 t")
    axes.set_ylabel("a(t)")
    axes.legend()


interactive(children=(FloatSlider(value=0.7, description='fracL', max=2.0999999999999996, min=-0.7), FloatSlid…